In [231]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [249]:
# creating df of data
csv_file = 'gym_members_exercise_tracking.csv'
data = pd.read_csv(csv_file)

features = data.drop(['Calories_Burned','Water_Intake (liters)', 'Workout_Frequency (days/week)'], axis=1)
target = data['Calories_Burned']

In [250]:
# encoding non numberical values
encoded_features = pd.get_dummies(features, columns=['Workout_Type', 'Gender'])
print(encoded_features)


# creating train and test dataframes
X_train, X_test, y_train, y_test = train_test_split(encoded_features, target, test_size=.2, random_state=42)

# scaling numerical values
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

     Age  Weight (kg)  Height (m)  Max_BPM  Avg_BPM  Resting_BPM  \
0     56         88.3        1.71      180      157           60   
1     46         74.9        1.53      179      151           66   
2     32         68.1        1.66      167      122           54   
3     25         53.2        1.70      190      164           56   
4     38         46.1        1.79      188      158           68   
..   ...          ...         ...      ...      ...          ...   
968   24         87.1        1.74      187      158           67   
969   25         66.6        1.61      184      166           56   
970   59         60.4        1.76      194      120           53   
971   32        126.4        1.83      198      146           62   
972   46         88.7        1.63      166      146           66   

     Session_Duration (hours)  Fat_Percentage  Experience_Level    BMI  \
0                        1.69            12.6                 3  30.20   
1                        1.30      

In [257]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(X_train_scaled.shape[1]),
    
    tf.keras.layers.Dense(256),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    
    tf.keras.layers.Dense(128),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    
    tf.keras.layers.Dense(64),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),

    tf.keras.layers.Dense(1)
])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=5,
                                                  restore_best_weights=True
                                                  )

optimizer = tf.keras.optimizers.Adam(learning_rate=.002)

model.compile(optimizer=optimizer,
                        loss='mse',
                        metrics=['mae']
                        )

history = model.fit(X_train_scaled,
                    y_train,
                    validation_split=.2,
                    validation_steps=10,
                    shuffle=True,
                    epochs=500,
                    callbacks=[early_stopping],
                    verbose=2
                    )

Epoch 1/500
20/20 - 1s - loss: 878707.6250 - mae: 898.0781 - val_loss: 919750.0625 - val_mae: 924.0025 - 1s/epoch - 51ms/step
Epoch 2/500
20/20 - 0s - loss: 874757.8750 - mae: 896.6710 - val_loss: 915157.0000 - val_mae: 922.2792 - 164ms/epoch - 8ms/step
Epoch 3/500
20/20 - 0s - loss: 871200.8125 - mae: 895.1048 - val_loss: 911017.0000 - val_mae: 920.5214 - 158ms/epoch - 8ms/step
Epoch 4/500
20/20 - 0s - loss: 867163.1875 - mae: 893.2742 - val_loss: 907012.6875 - val_mae: 918.6804 - 158ms/epoch - 8ms/step
Epoch 5/500
20/20 - 0s - loss: 862498.7500 - mae: 891.0775 - val_loss: 902256.4375 - val_mae: 916.4905 - 158ms/epoch - 8ms/step
Epoch 6/500
20/20 - 0s - loss: 857297.8750 - mae: 888.5947 - val_loss: 896304.1250 - val_mae: 913.8085 - 142ms/epoch - 7ms/step
Epoch 7/500
20/20 - 0s - loss: 851157.0625 - mae: 885.6730 - val_loss: 886407.8125 - val_mae: 909.2303 - 141ms/epoch - 7ms/step
Epoch 8/500
20/20 - 0s - loss: 844361.4375 - mae: 882.5709 - val_loss: 878215.0000 - val_mae: 905.2925 - 1

In [259]:
results = model.evaluate(X_test_scaled, y_test)
print("Test loss:", results[0])
print("Test mae:", results[1])

7/7 [==============================] - 0s 2ms/step - loss: 857.5059 - mae: 23.7712
Test loss: 857.5059204101562
Test mae: 23.77121353149414


In [285]:
# preparing prediction data
prediction_data = pd.read_csv('test_data.csv')
prediction_data = prediction_data.drop(['Water_Intake (liters)', 'Workout_Frequency (days/week)'], axis=1)

prediction_data = pd.get_dummies(prediction_data)

# preprocessing data
scaled_prediction_data = scaler.fit_transform(prediction_data)

scaled_predictions = model.predict(scaled_prediction_data)

print(scaled_predictions)


1/1 [==============================] - 0s 17ms/step
[[1332.731  ]
 [ 996.35364]
 [ 748.0112 ]
 [ 536.6415 ]
 [ 519.43054]
 [1187.3794 ]
 [1328.7806 ]
 [ 931.092  ]
 [ 723.9214 ]
 [ 800.0624 ]]


In [286]:
model.save('calorie_predictions.keras')